In [25]:
import streamlit as st
import pandas as pd
import numpy as np
import folium
from streamlit_folium import st_folium
import plotly.express as px

# Dashboard Setup
st.set_page_config(page_title="Air Pollution Dashboard", layout="wide")

# Load Data
@st.cache_data
def load_data():
    try:
        # Load the file you saved in Jupyter
        df = pd.read_csv('pollution_datasets_clean.csv')
        
        # Ensure numeric columns
        cols = ['Temperature', 'Humidity', 'PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'Air Quality']
        for col in cols:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        return df.dropna(subset=['Latitude', 'Longitude'])
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return None

df = load_data()

st.title("🌍 Air Pollution Monitoring Dashboard")

if df is not None:
    # Sidebar Filters
    st.sidebar.header("Filter Results")
    min_aqi = st.sidebar.slider("Minimum Air Quality Level", 0, 500, 0)
    filtered_data = df[df['Air Quality'] >= min_aqi]

    # Metrics Row
    col1, col2, col3 = st.columns(3)
    col1.metric("Average AQI", f"{filtered_data['Air Quality'].mean():.1f}")
    col2.metric("Total Locations", f"{len(filtered_data)}")
    col3.metric("Max PM2.5", f"{filtered_data['PM2.5'].max():.1f}")

    # Map Section
    st.subheader("📍 Interactive Pollution Map")
    m = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=4)
    for idx, row in filtered_data.head(100).iterrows():
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=5,
            color='red' if row['Air Quality'] > 150 else 'orange' if row['Air Quality'] > 100 else 'green',
            fill=True,
            popup=f"AQI: {row['Air Quality']}"
        ).add_to(m)
    st_folium(m, width=1000, height=500)

    # Chart Section
    st.subheader("📊 AQI Distribution")
    fig = px.histogram(filtered_data, x="Air Quality", nbins=30, color_discrete_sequence=['steelblue'])
    st.plotly_chart(fig, use_container_width=True)
else:
    st.warning("⚠️ Could not load data. Please ensure 'pollution_datasets_clean.csv' is in the same folder.")


2026-01-01 12:46:14.532 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2026-01-01 12:46:15.332 
  command:

    streamlit run C:\Python\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-01-01 12:46:15.346 No runtime found, using MemoryCacheStorageManager
